---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [6]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
test = df[:]
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [9]:
print(df.shape)
print(test.shape)

(500,)
(500,)


### Found 
- nums_only = 04/20/2009; 04/20/09; 4/20/09; 4/3/09
- alpha_present = 'Mar 20, 2009', 'March 20, 2009','20 Mar 2009','20 March 2009','Feb 2009','Sep 2009','Oct 2010'
'''

In [85]:
example = '''
Mar-20-2009
Mar. 20, 2009
Mar 20 2009
20 Mar. 2009
20 March, 2009
Mar 20th, 2009
Mar 21st, 2009
Mar 22nd, 2009

6/2008
12/2009
2009
2010'''

### INDICES TO INVESTIGATE
- 72 (mm-dd-yyyy): two present
- 178, 198, 199 (Jan, Feb, ... format) : returns empty list
- 

In [221]:
def count_empties(this_df):
    counter = 0
    for each in this_df:
        if (each == []) or (type(each) == str):
            counter+=1
    print('Dates Missing {}'.format(counter))
    found = this_df.shape[0] - counter
    print('Dates Found {}'.format(found))

In [490]:
# m(m)/d(d)/yy(yy), m(m)/d(d)/yy
# numbers only: 125
nums_only = r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}'
nums_only_df = test.str.findall(nums_only)

In [491]:
re.findall(nums_only,example)

[]

In [492]:
count_empties(nums_only_df)

Dates Missing 375
Dates Found 125


In [493]:
# inspect index 72
nums_only_df[101:126]

101      [10/05/97]
102    [07/18/2002]
103       [9/22/82]
104       [2/24/74]
105       [2/03/78]
106     [2/11/2006]
107       [8/22/83]
108       [5/04/74]
109     [7/20/2011]
110       [6/17/95]
111       [6/10/72]
112      [10/16/82]
113      [12/15/92]
114       [12/8/97]
115       [4/05/89]
116      [12/04/87]
117       [6/20/77]
118     [4/27/2006]
119      [07/17/92]
120      [12/22/98]
121      [10/02/96]
122      [11/05/90]
123       [5/04/77]
124       [2/27/96]
125              []
dtype: object

In [292]:
# to grab dates with Month at the beginning
#alpha_present = r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2}, )?\d{4}'
#alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_v2 = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Sept|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_present_df = df.str.findall(alpha_v2)
count_empties(alpha_present_df)

Dates Missing 282
Dates Found 218


In [293]:
alpha_present_df.loc[178]

['15 Jun 1985']

In [204]:
c = '''
Mar 20th, 2009
Mar-20-2009
Mar 20 2009
Mar 20th, 2009
Mar 21st, 2009
Mar 22nd, 2009
'''

In [290]:
alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
gottem = re.findall(alpha_present, b)
print(len(gottem))
gottem


11


['20 Mar. 2009',
 '20 March, 2009',
 'Mar 20th, 2009',
 'Mar-20-2009',
 'Mar. 20, 2009',
 'Mar 20 2009',
 '20 Mar. 2009',
 '20 March, 2009',
 'Mar 20th, 2009',
 'Mar 21st, 2009',
 'Mar 22nd, 2009']

In [266]:
count_empties(alpha_present_df)

Dates Missing 283
Dates Found 217


In [267]:
213 + 125

338

In [279]:
# in
alpha_present_df[178]

['15 Jun 1985']

In [108]:
def merge_found(x,y):
    if x == []:
        return y
    else:
        return x 

In [120]:
def insert_missing(got_it,missing):
    if len(got_it) == 0:
        return missing
        return got_it

In [300]:
found = alpha_present_df.combine(nums_only_df,merge_found)

In [318]:
single_dates = combined_df[missing]

In [320]:
#alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
nums_dates = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
#single_dates

(157,)

In [461]:
combined_df

0                                             [03/25/93]
1                                              [6/18/85]
2                                               [7/8/71]
3                                              [9/27/75]
4                                               [2/6/96]
5                                              [7/06/79]
6                                              [5/18/78]
7                                             [10/24/89]
8                                               [3/7/86]
9                                              [4/10/71]
10                                             [5/11/85]
11                                             [4/09/75]
12                                             [8/01/98]
13                                             [1/26/72]
14                                           [5/24/1990]
15                                           [1/25/2011]
16                                             [4/12/82]
17                             

In [562]:
nums_dates = r'\d{,2}[/]+\d{2,4}'
nums_dates_df = combined_df.str.findall(nums_dates)
cnt = 0
nums_dates_good_index = []
for entry,idx in zip(nums_dates_df,nums_dates_df.index.tolist()):
    if type(entry) == list:
        if len(entry) > 0:
            print(entry)
            cnt += 1
            nums_dates_good_index.append(idx)
print(cnt)

['6/1998']
['6/2005']
['10/1973']
['9/2005']
['03/1980']
['12/2005']
['5/1987']
['5/2004']
['8/1974']
['3/1986']
['10/1997']
['3/1993']
['3/1981']
['9/2003']
['10/1993']
['1/1983']
['7/1994']
['12/2008']
['10/1980']
['8/2003']
['12/1975']
['11/2010']
['7/1997']
['7/2014']
['9/2001']
['8/1986']
['1/1978']
['9/1975']
['1/2009']
['5/1995']
['8/1989']
['11/2000']
['06/1973']
['2/1999']
['6/2001']
['12/1978']
['8/2009']
['7/1973']
['1/2014']
['09/1975']
['12/2012']
['5/1999']
['7/1989']
['7/2009']
['11/1998']
['3/1995']
['2/2009']
['08/1988']
['4/2007']
['5/2000']
['4/2012']
['10/2001']
['2/1977']
['8/2008']
['2/1983']
['5/1979']
['1/1992']
['11/2008']
['12/2014']
['4/1974']
['10/1981']
['10/1986']
['03/1973']
['12/1994']
['8/1999']
['10/2010']
['10/1994']
['01/2007']
['8/2010']
['03/1990']
['11/2016']
['4/2004']
['2/1973']
['7/1987']
['8/2000']
['8/1975']
['5/1977']
['3/2000']
['9/1984']
['4/1973']
['12/1986']
['4/1979']
['7/2004']
['11/1984']
['5/2016']
['12/2007']
['11/1982']
['06/1981']

In [563]:
nums_dates_good_index

[343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454]

In [566]:
# add the newly found dates of the m(m)/(yy)yy format
combined_df[nums_dates_good_index] = nums_dates_df[nums_dates_good_index]

In [571]:
count_empties(combined_df)

Dates Missing 45
Dates Found 455


In [535]:
def not_null(a,b):
    if a.isnull():
        return b
    else:
        return a

In [524]:
def merge_missing(a,b):
    if type(a) == list:
        return a
    else:
        return b

In [544]:
count_empties(found)

Dates Missing 157
Dates Found 343


In [546]:
combined_df.iloc[343]

'6/1998 Primary Care Doctor:\n'

In [302]:
missing = []
found_idx = found.nonzero()
for each in found.index.tolist():
    if each not in found_idx[0]:
        missing.append(each)

In [304]:
# if the entry is a list, we have the answer.  i
# if the entry is a str it is missing
combined_df = found.combine(df,insert_missing)

In [454]:
count = 0
for each in combined_df:
    if type(each) != list:
        count += 1
count

157

In [455]:
df[missing].iloc[0]

'6/1998 Primary Care Doctor:\n'

In [263]:
alpha_v2 = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'

In [276]:
re.findall(alpha_v2,combined_df.loc[178])

['15 Jun 1985']

In [264]:
re.findall(alpha_v2,df[missing].iloc[0])

['15 Jun 1985']

In [89]:
short_nums_example = '''
6/2008
12/2009
2009
2010
'''

In [137]:
short_nums_only = r'(\d{,2}[/-]?\d{2,4})'
re.findall(short_nums_only,short_nums_example)

['6/2008', '12/2009', '2009', '2010']

In [139]:
def still_missing(x):
    if type(x) == list:
        return ""
    else:
        return x

In [141]:
temp_series = found2.apply(still_missing)

In [530]:
short_nums_df = temp_series.str.findall(short_nums_only)
found2[short_nums_df.nonzero()[0]]

178    15 Jun 1985 @ 11 AMCommunication with referrin...
198                                 [ October. 11, 2013]
199                                       [ Jan 24 1986]
204                                      [Mar. 10, 1976]
206                                   [ October 23 1990]
207                                    [ August 12 2004]
214                                     [ Sep. 10, 1974]
219                                       [ Dec 14 1975]
223                                   [ October 14 1974]
227                                [September. 15, 2011]
233                                        [ July, 1990]
234    . Patient presents with GAD flareup in Jul 200...
240                                         [ May, 2011]
244                                     [ January, 2013]
262    hHelen Ernst, M.D. last appt Jun 2002, no othe...
270                                          [May, 2006]
273                                    [ February, 2010]
306                            

In [488]:
print(test.iloc[499])

. In 1980, patient was living in Naples and described being upset by the amount of noise generated by the neighbor living under his apartment.  He noted the woman was a heroin user and when she refused to quiet down, he walked into her apartment and "trashed it".  He was arrested for breaking and entering and destruction of property and was placed on probation.



In [98]:
count_empties(short_nums_df)

Dates Missing 0
Dates Found 500


In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here